In [211]:
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import env
import os

In [222]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [223]:
def get_telco_data():
    filename = "telco_churn.csv"

    if os.path.isfile("telco_churn.csv"):
        return pd.read_csv("telco_churn.csv")
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql('''
SELECT *
FROM customers
JOIN contract_types ON contract_types.contract_type_id = customers.contract_type_id
JOIN internet_service_types ON internet_service_types.internet_service_type_id = customers.internet_service_type_id
JOIN payment_types ON payment_types.payment_type_id = customers.payment_type_id;
''', get_connection("telco_churn"))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv("telco_churn.csv")

        # Return the dataframe to the calling code
        return df 

In [224]:
df = get_telco_data()

In [225]:
df.head()

,Unnamed: 0,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check
1,1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,...,4,45.20,2460.55,No,3,Two year,1,DSL,4,Credit card (automatic)
2,2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,...,3,45.05,2560.1,No,3,Two year,1,DSL,3,Bank transfer (automatic)
3,3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,...,4,39.40,825.4,No,3,Two year,1,DSL,4,Credit card (automatic)
4,4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,...,3,85.15,6316.2,No,3,Two year,1,DSL,3,Bank transfer (automatic)


In [226]:
# contract type, gender, payment_type, autopay, internet_service_type

In [227]:
def clean_telco(df):
    df.total_charges.replace(to_replace = {" ":"0"}, inplace = True)
    df.total_charges = df.total_charges.astype("float")
    df["auto_pay"] = df.payment_type.str.contains("auto")
    df["auto_pay"] = df.auto_pay.replace(to_replace = [True,False],value = [1,0])
    dummy_df = pd.get_dummies(df[['gender', 'payment_type',"internet_service_type","contract_type"]])
    dummy_df.columns = [col.lower().replace(" ","_") for col in dummy_df]
    
    df["partner"] = df.partner.replace(to_replace = ["Yes","No"],value = [1,0])
    df["dependents"] = df.dependents.replace(to_replace = ["Yes","No"],value = [1,0])
    df["churn"] = df.churn.replace(to_replace = ["Yes","No"],value = [1,0])
    df["multiple_lines"] = df.paperless_billing.replace(to_replace = ["Yes","No","No phone service"],value = [1,0,0])
    df["paperless_billing"] = df.paperless_billing.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["tech_support"] = df.tech_support.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["device_protection"] = df.device_protection.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["online_backup"] = df.online_backup.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["online_security"] = df.online_security.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["streaming_tv"] = df.streaming_tv.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["streaming_movies"] = df.streaming_movies.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["phone_service"] = df.phone_service.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["multiple_lines"] = df.multiple_lines.replace(to_replace = ["Yes","No","No phone service"],value = [1,0,0])
    df["add_ons"] =  df.online_security + df.online_backup + df.device_protection + df.tech_support + df.streaming_tv + df.streaming_movies
    df.drop(columns = ["customer_id",
                       "gender",
                       "payment_type",
                       "internet_service_type",
                       "contract_type",
                   'internet_service_type_id',
                   "contract_type_id",
                   "payment_type_id",
                  "contract_type_id.1",
                   "internet_service_type_id.1",
                   "payment_type_id.1",
                   "Unnamed: 0"],inplace = True)
    return pd.concat([df, dummy_df], axis=1)
    

In [228]:
df = clean_telco(df)

In [230]:
df.auto_pay

0       0
1       1
2       1
3       1
4       1
       ..
7038    0
7039    0
7040    1
7041    0
7042    1
Name: auto_pay, Length: 7043, dtype: int64

In [221]:
df.add_ons

0       6
1       3
2       4
3       3
4       5
       ..
7038    0
7039    0
7040    0
7041    0
7042    0
Name: add_ons, Length: 7043, dtype: int64

In [208]:
df.columns

Index(['senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'auto_pay', 'gender_female', 'gender_male',
       'payment_type_bank_transfer_(automatic)',
       'payment_type_credit_card_(automatic)', 'payment_type_electronic_check',
       'payment_type_mailed_check', 'internet_service_type_dsl',
       'internet_service_type_fiber_optic', 'internet_service_type_none',
       'contract_type_month-to-month', 'contract_type_one_year',
       'contract_type_two_year', 'add_ons'],
      dtype='object')

In [198]:
df.dtypes

senior_citizen                              int64
partner                                     int64
dependents                                  int64
tenure                                      int64
phone_service                               int64
multiple_lines                              int64
online_security                             int64
online_backup                               int64
device_protection                           int64
tech_support                                int64
streaming_tv                                int64
streaming_movies                            int64
paperless_billing                           int64
monthly_charges                           float64
total_charges                             float64
churn                                       int64
auto_pay                                    int64
gender_Female                               uint8
gender_Male                                 uint8
payment_type_Bank_transfer_(automatic)      uint8


In [199]:
df.head()

,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,...,payment_type_Credit_card_(automatic),payment_type_Electronic_check,payment_type_Mailed_check,internet_service_type_DSL,internet_service_type_Fiber_optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One_year,contract_type_Two_year,add_ons
0,0,1,1,65,1,1,1,1,1,1,...,0,0,1,1,0,0,0,0,1,6
1,0,0,0,54,0,0,1,0,0,1,...,1,0,0,1,0,0,0,0,1,3
2,0,0,0,56,0,0,1,1,1,1,...,0,0,0,1,0,0,0,0,1,4
3,0,1,1,20,0,1,1,0,1,1,...,1,0,0,1,0,0,0,0,1,3
4,0,1,0,72,1,1,0,1,1,1,...,0,0,0,1,0,0,0,0,1,5


In [210]:
df.isnull().sum()

senior_citizen                            0
partner                                   0
dependents                                0
tenure                                    0
phone_service                             0
multiple_lines                            0
online_security                           0
online_backup                             0
device_protection                         0
tech_support                              0
streaming_tv                              0
streaming_movies                          0
paperless_billing                         0
monthly_charges                           0
total_charges                             0
churn                                     0
auto_pay                                  0
gender_female                             0
gender_male                               0
payment_type_bank_transfer_(automatic)    0
payment_type_credit_card_(automatic)      0
payment_type_electronic_check             0
payment_type_mailed_check       